## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

##  Xception架構

In [4]:
# 載入套件
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3)) # 定義輸入圖片大小
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
Xception_model = keras.applications.xception.Xception(include_top=False,  #是否包括頂部的全連接層。
                                                            weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
                                                            input_tensor= input_tensor, #Keras張量作為模型的輸入 
                                                            pooling=None, #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個4D張量。
                                                            classes=10) #圖片分類的類別數
'''Resnet 50 架構'''
#Resnet50_model=keras.applications.ResNet50(include_top=False, #不要導入全連接層，主要是由於輸出類別數量不同。
#                                  weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
#                                  input_tensor=input_tensor, #模型的輸入。
#                                  pooling=None,  #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個四維張量。
#                                  classes=10) #圖片分類的類別數量。

Xception_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

## 1.Resnet 50架構

In [9]:
# 載入套件
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3)) # 定義輸入圖片大小
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
#Xception_model = keras.keras.applications.xception.Xception(include_top=False,  #是否包括頂部的全連接層。
#                                                            weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
#                                                            input_tensor= input_tensor, #Keras張量作為模型的輸入 
#                                                            pooling=None, #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個4D張量。
#                                                            classes=10) #圖片分類的類別數
'''Resnet 50 架構'''
Resnet50_model=keras.applications.ResNet50(include_top=False, #不要導入全連接層，主要是由於輸出類別數量不同。
                                  weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
                                  input_tensor=input_tensor, #模型的輸入。
                                  pooling=None,  #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個四維張量。
                                  classes=10) #圖片分類的類別數量。
#輸出Resnet 50架構
print(Resnet50_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## 2.添加層數

In [11]:
#接上原本的output
x =Resnet50_model.output
#自行新增新的層
x = GlobalAveragePooling2D()(x) #對空間數據(2D)新增平均池化層
x = Dense(output_dim=128, activation='relu')(x) #
x = Dropout(p=0.1)(x)
'''可以參考Cifar10實作章節'''

#重新定義輸出類別的數量
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=Resnet50_model.input, outputs=predictions)
#觀看模型的深度(此行目的是接著從模型深度來決定凍結的層數量)
print('ResNet_Model深度：', len(model.layers))


ResNet_Model深度： 179


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # Remove the CWD from sys.path while we load stuff.


## 3. 鎖定特定幾層不要更新權重

In [12]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [13]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [14]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 140s 3ms/step - loss: 1.2391 - acc: 0.5845
Epoch 2/100
50000/50000 [==============================] - 130s 3ms/step - loss: 1.0017 - acc: 0.6581
Epoch 3/100
50000/50000 [==============================] - 131s 3ms/step - loss: 0.8926 - acc: 0.6947
Epoch 4/100
50000/50000 [==============================] - 130s 3ms/step - loss: 0.8221 - acc: 0.7183
Epoch 5/100
50000/50000 [==============================] - 130s 3ms/step - loss: 0.7287 - acc: 0.7452
Epoch 6/100
50000/50000 [==============================] - 131s 3ms/step - loss: 0.6442 - acc: 0.7769
Epoch 7/100
50000/50000 [==============================] - 131s 3ms/step - loss: 0.5800 - acc: 0.7916
Epoch 8/100
50000/50000 [==============================] - 130s 3ms/step - loss: 0.5406 - acc: 0.8118
Epoch 9/100
50000/50000 [==============================] - 128s 3ms/step - loss: 0.4893 -

50000/50000 [==============================] - 128s 3ms/step - loss: 0.1356 - acc: 0.9740
Epoch 76/100
50000/50000 [==============================] - 126s 3ms/step - loss: 0.1014 - acc: 0.9793
Epoch 77/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1025 - acc: 0.9788
Epoch 78/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1296 - acc: 0.9742
Epoch 79/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1296 - acc: 0.9737
Epoch 80/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1061 - acc: 0.9803
Epoch 81/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.2178 - acc: 0.9690
Epoch 82/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.2078 - acc: 0.9739
Epoch 83/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.2181 - acc: 0.9742
Epoch 84/100
50000/50000 [==============================] - 127s 3ms/step - loss: 0.16

# 儲存模型

儲存訓練好的模型，格式為.h5

In [15]:
model.save('Day021_Transfer Learning_HW.h5')